In [2]:
import numpy as np
from cvxopt import matrix
from cvxopt import solvers

In [3]:
train_file = "C:/IITD/sem5/col774-ml/datasets/fmnist_data/fashion_mnist/train.csv"
test_file = "C:/IITD/sem5/col774-ml/datasets/fmnist_data/fashion_mnist/test.csv"
val_file = "C:/IITD/sem5/col774-ml/datasets/fmnist_data/fashion_mnist/val.csv"

train_data = np.genfromtxt(train_file, delimiter=',')
test_data = np.genfromtxt(test_file, delimiter=',')
val_data = np.genfromtxt(val_file, delimiter=',')

In [5]:
def load_data(data_file):
    data = np.genfromtxt(data_file, delimiter=',')
    
    d = 4 # last digit of entry number
    data = data[(data[:,-1] == d) | (data[:, -1] == d+1)]   # filter for labels d and d+1

    m = data.shape[0]
    x = data[:, :-1]  # features
    x /= x.max()    # scale to 0-1
    y = data[:, -1].reshape((m, 1))  # labels
    y = np.where(y == d, -1, 1) # change label d to -1 and d+1 to 1

    return m, x, y

m, x, y = load_data(train_file)
m_test, x_test, y_test = load_data(test_file)
m_val, x_val, y_val = load_data(val_file)

In [9]:
P = np.zeros((m, m))

for i in range(m):
    if i % 100 == 0:
        print(i)
    for j in range(i, m):
        P[i][j] = y[i] * y[j] * np.dot(x[i], x[j])
        P[j][i] = P[i][j]

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400


In [10]:
q = -np.ones((m, 1))

G = np.vstack((-np.identity(m), np.identity(m)))
C = 1
h = np.vstack((np.zeros((m, 1)), np.full((m, 1), C)))

A = y.T
b = np.zeros(1)
print(A.dtype, b.dtype, q.dtype, G.dtype, h.dtype)

int32 float64 float64 float64 float64


In [11]:
P = matrix(P, tc='d')
q = matrix(q, tc='d')
G = matrix(G, tc='d')
h = matrix(h, tc='d')
A = matrix(A, tc='d')
b = matrix(b, tc='d')

In [12]:
sol = solvers.qp(P,q,G,h,A,b)

     pcost       dcost       gap    pres   dres
 0: -1.8144e+02 -7.2340e+03  4e+04  2e+00  2e-12
 1: -1.0203e+02 -3.4492e+03  6e+03  3e-01  9e-13
 2: -3.3534e+01 -9.1974e+02  2e+03  7e-02  7e-13
 3: -1.1925e+01 -4.1859e+02  7e+02  2e-02  3e-13
 4: -3.6847e+00 -1.7770e+02  3e+02  8e-03  1e-13
 5: -1.2594e+00 -5.9907e+01  9e+01  2e-03  4e-14
 6: -6.7211e-01 -2.5705e+01  4e+01  9e-04  2e-14
 7: -4.1327e-01 -1.1925e+01  2e+01  3e-04  1e-14
 8: -5.1263e-01 -5.5495e+00  6e+00  1e-04  8e-15
 9: -5.7700e-01 -3.1645e+00  3e+00  4e-05  8e-15
10: -7.0634e-01 -2.0084e+00  1e+00  3e-06  9e-15
11: -9.3593e-01 -1.4177e+00  5e-01  7e-07  1e-14
12: -1.0291e+00 -1.2146e+00  2e-01  2e-16  1e-14
13: -1.0863e+00 -1.1319e+00  5e-02  3e-16  1e-14
14: -1.1048e+00 -1.1100e+00  5e-03  3e-16  1e-14
15: -1.1073e+00 -1.1074e+00  1e-04  2e-16  1e-14
16: -1.1073e+00 -1.1073e+00  1e-06  5e-16  1e-14
17: -1.1073e+00 -1.1073e+00  1e-08  8e-16  1e-14
Optimal solution found.


In [13]:
alpha = np.array(sol['x'])

w = sum(alpha[i] * y[i] * x[i] for i in range(m))

S = set() # support vectors
for i in range(m):
    if alpha[i] > 1e-3:
        S.add(i)

b = sum(y[s] - np.dot(w, x[s]) for s in S) / len(S) # take average over all support vectors

In [15]:
b

array([0.49688294])

In [229]:
def indicator(exp):
    if exp:
        return 1
    else:
        return 0

train_acc = sum(indicator(y[i] == np.sign(np.dot(w, x[i]) + b)) for i in range(m)) / m
val_acc = sum(indicator(y_val[i] == np.sign(np.dot(w, x_val[i]) + b)) for i in range(m_val)) / m_val
test_acc = sum(indicator(y_test[i] == np.sign(np.dot(w, x_test[i]) + b)) for i in range(m_test)) / m_test

print(train_acc, val_acc, test_acc)
# for i in range(m_test):
#     print(y_test[i], np.sign(np.dot(w, x_test[i]) + b))

1.0 0.996 0.998


In [232]:
def gaussian_kernel(x, z, gamma):
    return np.exp(-gamma * np.linalg.norm(x-z)**2)

In [234]:
gamma = 0.05

P = np.zeros((m, m))

for i in range(m):
    if i % 100 == 0:
        print(i)
    for j in range(i, m):
        P[i][j] = y[i] * y[j] * gaussian_kernel(x[i], x[j], gamma)
        P[j][i] = P[i][j]

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400


In [7]:
q = -np.ones((m, 1))
G = np.vstack((-np.identity(m), np.identity(m)))
C = 1
h = np.vstack((np.zeros((m, 1)), np.full((m, 1), C)))
A = y.T
b = np.zeros(1)

P = matrix(P, tc='d')
q = matrix(q, tc='d')
G = matrix(G, tc='d')
h = matrix(h, tc='d')
A = matrix(A, tc='d')
b = matrix(b, tc='d')

sol = solvers.qp(P,q,G,h,A,b)

     pcost       dcost       gap    pres   dres
 0: -1.8144e+02 -7.2340e+03  4e+04  2e+00  2e-12
 1: -1.0203e+02 -3.4492e+03  6e+03  3e-01  9e-13
 2: -3.3534e+01 -9.1974e+02  2e+03  7e-02  7e-13
 3: -1.1925e+01 -4.1859e+02  7e+02  2e-02  3e-13
 4: -3.6847e+00 -1.7770e+02  3e+02  8e-03  1e-13
 5: -1.2594e+00 -5.9907e+01  9e+01  2e-03  4e-14
 6: -6.7211e-01 -2.5705e+01  4e+01  9e-04  2e-14
 7: -4.1327e-01 -1.1925e+01  2e+01  3e-04  1e-14
 8: -5.1263e-01 -5.5495e+00  6e+00  1e-04  8e-15
 9: -5.7700e-01 -3.1645e+00  3e+00  4e-05  8e-15
10: -7.0634e-01 -2.0084e+00  1e+00  3e-06  9e-15
11: -9.3593e-01 -1.4177e+00  5e-01  7e-07  1e-14
12: -1.0291e+00 -1.2146e+00  2e-01  2e-16  1e-14
13: -1.0863e+00 -1.1319e+00  5e-02  3e-16  1e-14
14: -1.1048e+00 -1.1100e+00  5e-03  3e-16  1e-14
15: -1.1073e+00 -1.1074e+00  1e-04  2e-16  1e-14
16: -1.1073e+00 -1.1073e+00  1e-06  5e-16  1e-14
17: -1.1073e+00 -1.1073e+00  1e-08  8e-16  1e-14
Optimal solution found.


In [8]:
alpha = np.array(sol['x'])

S = set() # support vectors
for i in range(m):
    if alpha[i] > 1e-3:
        S.add(i)

b = sum(y[s] - sum(alpha[j] * y[j] * gaussian_kernel(x[j], x[s], gamma) for j in S) for s in S) / len(S) # take average over all support vectors


NameError: name 'gaussian_kernel' is not defined

In [266]:
def predict(x_predict):
    return np.sign(sum(alpha[i] * y[i] * gaussian_kernel(x[i], x_predict, gamma) for i in range(m)) + b)

val_acc = sum(indicator(y_val[i] == predict(x_val[i])) for i in range(m_val)) / m_val
test_acc = sum(indicator(y_test[i] == predict(x_test[i])) for i in range(m_test)) / m_test

print(val_acc, test_acc)

0.996 0.996
